In [54]:
# Import essential libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

In [55]:
pd.set_option('display.max_columns', None)

# Load the LRI AI Spreadsheet data
df = pd.read_csv('data/LRI_AI_Spreadsheet_WVI_NG_11_2025_Deidentified.csv')

# Rename Pentagram columns to Pentacam
df = df.rename(columns={
    '∆k Pentagram magnitude (D)': '∆k Pentacam magnitude (D)',
    '∆k Pentagram axis (°)': '∆k Pentacam axis (°)'
})
print("Renamed 'Pentagram' columns to 'Pentacam'")

Renamed 'Pentagram' columns to 'Pentacam'


In [56]:
# Function to convert astigmatism magnitude + axis to magnitude at a specific axis
def get_magnitude_at_axis(magnitude: float, angle_degrees: float, desired_axis_degrees: float) -> float:
    """
    Convert a single astigmatism measurement to its magnitude at a specified axis.

    Args:
        magnitude: Original magnitude of astigmatism (in diopters, positive cylinder)
        angle_degrees: Original axis of astigmatism (in degrees)
        desired_axis_degrees: Axis at which to calculate the magnitude (in degrees)

    Returns:
        float: Magnitude at the desired axis (in diopters)
    """
    if magnitude == 0:
        return 0.0
    
    # Convert to double-angle formula
    angle_rad = np.radians(2 * (desired_axis_degrees - angle_degrees))
    
    # Calculate magnitude at desired axis using double-angle formula
    magnitude_at_desired_axis = magnitude * np.cos(angle_rad)
    
    return magnitude_at_desired_axis

def convert_neg_to_pos_cyl(neg_cyl: float, neg_axis: float) -> tuple:
    """
    Convert negative cylinder notation to positive cylinder notation.
    
    Args:
        neg_cyl: Cylinder value in negative format (negative number)
        neg_axis: Axis in negative cylinder notation (degrees)
    
    Returns:
        tuple: (positive_cylinder, converted_axis)
    """
    # Flip the sign to get positive cylinder
    pos_cyl = -neg_cyl
    
    # Add or subtract 90 from axis, keeping within 1-180 range
    converted_axis = neg_axis + 90
    if converted_axis > 180:
        converted_axis -= 180
    
    return pos_cyl, converted_axis

# Create new column: (M) Cylinder at Barrett Integrated-K axis
# First convert from negative to positive cylinder format, then apply formula
def calc_neg_cylinder_at_bik_axis(row):
    neg_cyl = row['(M) Cylinder']
    neg_axis = row['(M) Axis']
    bik_axis = row['Barrett Integrated-K axis']
    
    # Convert to positive cylinder format
    pos_cyl, converted_axis = convert_neg_to_pos_cyl(neg_cyl, neg_axis)
    
    # Calculate magnitude at Barrett Integrated-K axis
    return get_magnitude_at_axis(pos_cyl, converted_axis, bik_axis)

df['(M) Cylinder_atBIKaxis'] = df.apply(calc_neg_cylinder_at_bik_axis, axis=1)

# Reorder columns to place new column next to (M) Cylinder
cols = df.columns.tolist()
cyl_idx = cols.index('(M) Cylinder')
# Remove the new column from its current position and insert after (M) Cylinder
cols.remove('(M) Cylinder_atBIKaxis')
cols.insert(cyl_idx + 1, '(M) Cylinder_atBIKaxis')
df = df[cols]

# Also create intermediate columns to show the conversion (optional - can remove later)
df['(M) Cyl_posCyl'] = -df['(M) Cylinder']
df['(M) Axis_posCyl'] = df['(M) Axis'].apply(lambda x: x + 90 if x + 90 <= 180 else x + 90 - 180)

print("Created new column '(M) Cylinder_atBIKaxis'")
print("\nConversion: Negative Cyl → Positive Cyl → Magnitude at BIK axis")
print(f"\nSample values:")
df[['(M) Cylinder', '(M) Axis', '(M) Cyl_posCyl', '(M) Axis_posCyl', 'Barrett Integrated-K axis', '(M) Cylinder_atBIKaxis']].head(10)


Created new column '(M) Cylinder_atBIKaxis'

Conversion: Negative Cyl → Positive Cyl → Magnitude at BIK axis

Sample values:


,(M) Cylinder,(M) Axis,(M) Cyl_posCyl,(M) Axis_posCyl,Barrett Integrated-K axis,(M) Cylinder_atBIKaxis
0,-0.50,74,0.50,164,12,0.279596
1,-0.25,76,0.25,166,173,0.242574
2,-0.75,151,0.75,61,82,0.557359
3,-0.75,166,0.75,76,93,0.621778
4,-0.50,121,0.50,31,24,0.485148
5,-0.25,80,0.25,170,165,0.246202
6,-0.25,142,0.25,52,81,0.132480
7,-0.50,175,0.50,85,82,0.497261
8,-0.50,90,0.50,180,12,0.456773
9,-0.50,135,0.50,45,68,0.347329


In [57]:
# Calculate magnitude at BIK axis for other astigmatism measurements (already in positive cylinder format)

# 1. Pentacam ∆k at BIK axis
df['Pentacam_atBIKaxis'] = df.apply(
    lambda row: get_magnitude_at_axis(
        row['∆k Pentacam magnitude (D)'], 
        row['∆k Pentacam axis (°)'], 
        row['Barrett Integrated-K axis']
    ) if pd.notna(row['∆k Pentacam magnitude (D)']) else np.nan, 
    axis=1
)

# 2. ∆k IOL 700 at BIK axis
df['deltaK_IOL700_atBIKaxis'] = df.apply(
    lambda row: get_magnitude_at_axis(
        row['∆k IOL 700 magnitude (D)'], 
        row['∆k IOL 700 axis (°)'], 
        row['Barrett Integrated-K axis']
    ) if pd.notna(row['∆k IOL 700 magnitude (D)']) else np.nan, 
    axis=1
)

# 3. Post. astigmatism (IOL 700) at BIK axis
df['PostAstig_IOL700_atBIKaxis'] = df.apply(
    lambda row: get_magnitude_at_axis(
        row['Post. astigmatism (IOL 700) magnitude (D)'], 
        row['Post. astigmatism (IOL 700) axis (°)'], 
        row['Barrett Integrated-K axis']
    ) if pd.notna(row['Post. astigmatism (IOL 700) magnitude (D)']) else np.nan, 
    axis=1
)

# 4. ∆TK IOL 700 at BIK axis
df['deltaTK_IOL700_atBIKaxis'] = df.apply(
    lambda row: get_magnitude_at_axis(
        row['∆TK IOL 700 magnitude (D)'], 
        row['∆TK IOL 700 axis (°)'], 
        row['Barrett Integrated-K axis']
    ) if pd.notna(row['∆TK IOL 700 magnitude (D)']) else np.nan, 
    axis=1
)

print("Created 4 new columns for magnitude at BIK axis:")
print("  1. Pentacam_atBIKaxis")
print("  2. deltaK_IOL700_atBIKaxis")
print("  3. PostAstig_IOL700_atBIKaxis")
print("  4. deltaTK_IOL700_atBIKaxis")

# Show sample of all new columns
print("\nSample values:")
df[['Barrett Integrated-K axis', 
    '∆k Pentacam magnitude (D)', '∆k Pentacam axis (°)', 'Pentacam_atBIKaxis',
    '∆k IOL 700 magnitude (D)', '∆k IOL 700 axis (°)', 'deltaK_IOL700_atBIKaxis']].head(10)


Created 4 new columns for magnitude at BIK axis:
  1. Pentacam_atBIKaxis
  2. deltaK_IOL700_atBIKaxis
  3. PostAstig_IOL700_atBIKaxis
  4. deltaTK_IOL700_atBIKaxis

Sample values:


,Barrett Integrated-K axis,∆k Pentacam magnitude (D),∆k Pentacam axis (°),Pentacam_atBIKaxis,∆k IOL 700 magnitude (D),∆k IOL 700 axis (°),deltaK_IOL700_atBIKaxis
0,12,0.3,21.7,0.282967,0.57,7.0,0.561340
1,173,0.4,144.9,0.222518,0.64,9.0,0.542751
2,82,1.4,79.9,1.396240,1.76,84.0,1.755713
3,93,1.3,96.9,1.287972,1.83,90.0,1.819975
4,24,0.7,22.0,0.698295,0.62,26.0,0.618490
5,165,0.7,162.5,0.697336,0.70,168.0,0.696165
6,81,1.1,78.6,1.096142,1.12,83.0,1.117272
7,82,1.3,79.1,1.293345,1.60,84.0,1.596102
8,12,0.0,145.3,0.000000,0.22,12.0,0.220000
9,68,0.5,54.7,0.447077,0.50,84.0,0.424024


In [58]:
# Show sample for Post. astigmatism and ∆TK columns
print("Post. astigmatism and ∆TK IOL 700 at BIK axis:\n")
df[['Barrett Integrated-K magnitude (D)','Barrett Integrated-K axis', 
    'Post. astigmatism (IOL 700) magnitude (D)', 'Post. astigmatism (IOL 700) axis (°)', 'PostAstig_IOL700_atBIKaxis',
    '∆TK IOL 700 magnitude (D)', '∆TK IOL 700 axis (°)', 'deltaTK_IOL700_atBIKaxis']].head(10)


Post. astigmatism and ∆TK IOL 700 at BIK axis:



,Barrett Integrated-K magnitude (D),Barrett Integrated-K axis,Post. astigmatism (IOL 700) magnitude (D),Post. astigmatism (IOL 700) axis (°),PostAstig_IOL700_atBIKaxis,∆TK IOL 700 magnitude (D),∆TK IOL 700 axis (°),deltaTK_IOL700_atBIKaxis
0,0.42,12,0.00,90.0,0.000000,0.65,7.0,0.640125
1,0.38,173,0.04,12.0,0.031520,0.77,9.0,0.652997
2,1.60,82,0.42,0.0,-0.403730,1.59,82.0,1.590000
3,1.57,93,0.48,8.0,-0.472708,1.63,87.0,1.594381
4,0.66,24,0.15,179.0,0.096418,0.81,22.0,0.808027
5,0.67,165,0.11,22.0,0.030320,0.85,171.0,0.831425
6,1.11,81,0.41,8.0,-0.339905,0.95,77.0,0.940755
7,1.45,82,0.23,178.0,-0.224974,1.60,83.0,1.599025
8,0.11,12,0.21,170.0,0.151061,0.42,2.0,0.394671
9,0.48,68,0.20,18.0,-0.034730,0.46,74.0,0.449948


In [59]:
# Create a special term for the Barrett Integrated-K axis called "BIK_axis_term" to better capture the information held by the angle
# Using double-angle cosine: cos(2*axis) converts 180° periodic axis to linear term
# +1 = axis 0°/180° (with-the-rule), -1 = axis 90° (against-the-rule), 0 = axis 45°/135° (oblique)

df['BIK_axis_term'] = np.cos(np.radians(df['Barrett Integrated-K axis'] * 2))

print("Created 'BIK_axis_term' column")
print("\nSample values:")
df[['Barrett Integrated-K axis', 'BIK_axis_term']].head(10)


Created 'BIK_axis_term' column

Sample values:


,Barrett Integrated-K axis,BIK_axis_term
0,12,0.913545
1,173,0.970296
2,82,-0.961262
3,93,-0.994522
4,24,0.669131
5,165,0.866025
6,81,-0.951057
7,82,-0.961262
8,12,0.913545
9,68,-0.719340


In [60]:
# SET RANDOM_STATE AND SHUFFLE THE DATASET ('df')
df = shuffle(df, random_state=42)

In [61]:
#specify the categorical variables
df['Laterality'] = df['Laterality'].astype('category')
df['Single/Paired'] = df['Single/Paired'].astype('category')

In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1425 entries, 628 to 1126
Data columns (total 32 columns):
 #   Column                                     Non-Null Count  Dtype   
---  ------                                     --------------  -----   
 0   Identifier                                 1425 non-null   object  
 1   Age                                        1425 non-null   int64   
 2   Laterality                                 1425 non-null   category
 3   Procedure                                  1425 non-null   int64   
 4   (M) Sphere                                 1425 non-null   float64 
 5   (M) Cylinder                               1425 non-null   float64 
 6   (M) Cylinder_atBIKaxis                     1425 non-null   float64 
 7   (M) Axis                                   1425 non-null   int64   
 8   ∆k Pentacam magnitude (D)                  1421 non-null   float64 
 9   ∆k Pentacam axis (°)                       1425 non-null   float64 
 10  ∆k IOL 700 magn

In [65]:
# Setting up features and target for first model (to predict number of arcuates)
target1 = ['Single/Paired']

# Setting up features for first model
features1 = ['Age', 'Laterality', 'Barrett Integrated-K magnitude (D)', 'BIK_axis_term', 'deltaTK_IOL700_atBIKaxis', '(M) Cylinder_atBIKaxis']